In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, fbeta_score, recall_score
from sklearn.svm import SVC

In [2]:
lulc = pd.read_csv("lulc2013-2020.ascii",delim_whitespace=" ",header=None)
NDVI = pd.read_csv("ndvi2013-2020.ascii",delim_whitespace=" ",header=None)
RD = pd.read_csv("road2013-2020.ascii",delim_whitespace=" ",header=None)
NTL = pd.read_csv("light2013-2020.ascii",delim_whitespace=" ",header=None)
NLST = pd.read_csv("NLST_WinterHalf_13to20.txt",delim_whitespace=" ",header=None)

features = ['LAT', 'LON', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

lulc.columns=features
NDVI.columns=features
RD.columns=features
NTL.columns=features
NLST.columns=features

label = [0] * 26458

NDVI['LABEL2013'] = label
NDVI['LABEL2014'] = label
NDVI['LABEL2015'] = label
NDVI['LABEL2016'] = label
NDVI['LABEL2017'] = label
NDVI['LABEL2018'] = label
NDVI['LABEL2019'] = label
NDVI['LABEL2020'] = label

NTL['LABEL2013'] = label
NTL['LABEL2014'] = label
NTL['LABEL2015'] = label
NTL['LABEL2016'] = label
NTL['LABEL2017'] = label
NTL['LABEL2018'] = label
NTL['LABEL2019'] = label
NTL['LABEL2020'] = label

RD['LABEL2013'] = label
RD['LABEL2014'] = label
RD['LABEL2015'] = label
RD['LABEL2016'] = label
RD['LABEL2017'] = label
RD['LABEL2018'] = label
RD['LABEL2019'] = label
RD['LABEL2020'] = label

NLST['LABEL2013'] = label
NLST['LABEL2014'] = label
NLST['LABEL2015'] = label
NLST['LABEL2016'] = label
NLST['LABEL2017'] = label
NLST['LABEL2018'] = label
NLST['LABEL2019'] = label
NLST['LABEL2020'] = label

lulc5class = lulc[['LAT', 'LON']].copy()
lulc5class['2013'] = label
lulc5class['2014'] = label
lulc5class['2015'] = label
lulc5class['2016'] = label
lulc5class['2017'] = label
lulc5class['2018'] = label
lulc5class['2019'] = label
lulc5class['2020'] = label

In [3]:

for i in range(26458):
    for year in ['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']:
        value = lulc[year].loc[i]
        if value in [1, 2, 3, 4, 5, 6, 7, 8, 9]:
            lulc5class[year].loc[i] = 1
        elif value == 10:
            lulc5class[year].loc[i] = 2
        elif value in [11, 15, 17]:
            lulc5class[year].loc[i] = 3
            NDVI['LABEL'+year].loc[i] = 1
            NTL['LABEL'+year].loc[i] = 1
            RD['LABEL'+year].loc[i] = 1
            NLST['LABEL'+year].loc[i] = 1
        elif value == 12 or value == 14:
            lulc5class[year].loc[i] = 4
        elif value == 13:
            lulc5class[year].loc[i] = 5
            NDVI['LABEL'+year].loc[i] = 4
            NTL['LABEL'+year].loc[i] = 4
            RD['LABEL'+year].loc[i] = 4
            NLST['LABEL'+year].loc[i] = 4
        elif value == 16:
            lulc5class[year].loc[i] = 6

/Applications/anaconda3/envs/ML_env/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [4]:
NDVI_2013 = NDVI[['LAT','LON','2013','LABEL2013']].copy() 
RD_2013 = RD[['LAT','LON','2013','LABEL2013']].copy() 
NTL_2013 = NTL[['LAT','LON','2013','LABEL2013']].copy() 
NLST_2013 = NLST[['LAT','LON','2013','LABEL2013']].copy() 
NDVI_2013_Urban = NDVI_2013[(NDVI_2013['LABEL2013'] == 4)]
RD_2013_Urban = RD_2013[(RD_2013['LABEL2013'] == 4)]
NTL_2013_Urban = NTL_2013[(NTL_2013['LABEL2013'] == 4)]
NLST_2013_Urban = NLST_2013[(NLST_2013['LABEL2013'] == 4)]
NDVI_2013_Water = NDVI_2013[(NDVI_2013['LABEL2013'] == 1)]
RD_2013_Water = RD_2013[(RD_2013['LABEL2013'] == 1)]
NTL_2013_Water = NTL_2013[(NTL_2013['LABEL2013'] == 1)]
NLST_2013_Water = NLST_2013[(NLST_2013['LABEL2013'] == 1)]
NDVI_2013_NotUrban = NDVI_2013[(NDVI_2013['LABEL2013'] != 4)]
RD_2013_NotUrban = RD_2013[(RD_2013['LABEL2013'] != 4)]
NTL_2013_NotUrban = NTL_2013[(NTL_2013['LABEL2013'] != 4)]
NLST_2013_NotUrban = NLST_2013[(NLST_2013['LABEL2013'] != 4)]
NDVI_2013_NotUrbanNotWater = NDVI_2013_NotUrban[(NDVI_2013_NotUrban['LABEL2013'] != 1)]
RD_2013_NotUrbanNotWater = RD_2013_NotUrban[(RD_2013_NotUrban['LABEL2013'] != 1)]
NTL_2013_NotUrbanNotWater = NTL_2013_NotUrban[(NTL_2013_NotUrban['LABEL2013'] != 1)]
NLST_2013_NotUrbanNotWater = NLST_2013_NotUrban[(NLST_2013_NotUrban['LABEL2013'] != 1)]

NDVI_2014 = NDVI[['LAT','LON','2014','LABEL2014']].copy() 
RD_2014 = RD[['LAT','LON','2014','LABEL2014']].copy() 
NTL_2014 = NTL[['LAT','LON','2014','LABEL2014']].copy() 
NLST_2014 = NLST[['LAT','LON','2014','LABEL2014']].copy() 
NDVI_2014_Urban = NDVI_2014[(NDVI_2014['LABEL2014'] == 4)]
RD_2014_Urban = RD_2014[(RD_2014['LABEL2014'] == 4)]
NTL_2014_Urban = NTL_2014[(NTL_2014['LABEL2014'] == 4)]
NLST_2014_Urban = NLST_2014[(NLST_2014['LABEL2014'] == 4)]
NDVI_2014_Water = NDVI_2014[(NDVI_2014['LABEL2014'] == 1)]
RD_2014_Water = RD_2014[(RD_2014['LABEL2014'] == 1)]
NTL_2014_Water = NTL_2014[(NTL_2014['LABEL2014'] == 1)]
NLST_2014_Water = NLST_2014[(NLST_2014['LABEL2014'] == 1)]
NDVI_2014_NotUrban = NDVI_2014[(NDVI_2014['LABEL2014'] != 4)]
RD_2014_NotUrban = RD_2014[(RD_2014['LABEL2014'] != 4)]
NTL_2014_NotUrban = NTL_2014[(NTL_2014['LABEL2014'] != 4)]
NLST_2014_NotUrban = NLST_2014[(NLST_2014['LABEL2014'] != 4)]
NDVI_2014_NotUrbanNotWater = NDVI_2014_NotUrban[(NDVI_2014_NotUrban['LABEL2014'] != 1)]
RD_2014_NotUrbanNotWater = RD_2014_NotUrban[(RD_2014_NotUrban['LABEL2014'] != 1)]
NTL_2014_NotUrbanNotWater = NTL_2014_NotUrban[(NTL_2014_NotUrban['LABEL2014'] != 1)]
NLST_2014_NotUrbanNotWater = NLST_2014_NotUrban[(NLST_2014_NotUrban['LABEL2014'] != 1)]

NDVI_2015 = NDVI[['LAT','LON','2015','LABEL2015']].copy() 
RD_2015 = RD[['LAT','LON','2015','LABEL2015']].copy() 
NTL_2015 = NTL[['LAT','LON','2015','LABEL2015']].copy() 
NLST_2015 = NLST[['LAT','LON','2015','LABEL2015']].copy() 
NDVI_2015_Urban = NDVI_2015[(NDVI_2015['LABEL2015'] == 4)]
RD_2015_Urban = RD_2015[(RD_2015['LABEL2015'] == 4)]
NTL_2015_Urban = NTL_2015[(NTL_2015['LABEL2015'] == 4)]
NLST_2015_Urban = NLST_2015[(NLST_2015['LABEL2015'] == 4)]
NDVI_2015_Water = NDVI_2015[(NDVI_2015['LABEL2015'] == 1)]
RD_2015_Water = RD_2015[(RD_2015['LABEL2015'] == 1)]
NTL_2015_Water = NTL_2015[(NTL_2015['LABEL2015'] == 1)]
NLST_2015_Water = NLST_2015[(NLST_2015['LABEL2015'] == 1)]
NDVI_2015_NotUrban = NDVI_2015[(NDVI_2015['LABEL2015'] != 4)]
RD_2015_NotUrban = RD_2015[(RD_2015['LABEL2015'] != 4)]
NTL_2015_NotUrban = NTL_2015[(NTL_2015['LABEL2015'] != 4)]
NLST_2015_NotUrban = NLST_2015[(NLST_2015['LABEL2015'] != 4)]
NDVI_2015_NotUrbanNotWater = NDVI_2015_NotUrban[(NDVI_2015_NotUrban['LABEL2015'] != 1)]
RD_2015_NotUrbanNotWater = RD_2015_NotUrban[(RD_2015_NotUrban['LABEL2015'] != 1)]
NTL_2015_NotUrbanNotWater = NTL_2015_NotUrban[(NTL_2015_NotUrban['LABEL2015'] != 1)]
NLST_2015_NotUrbanNotWater = NLST_2015_NotUrban[(NLST_2015_NotUrban['LABEL2015'] != 1)]

NDVI_2016 = NDVI[['LAT','LON','2016','LABEL2016']].copy() 
RD_2016 = RD[['LAT','LON','2016','LABEL2016']].copy() 
NTL_2016 = NTL[['LAT','LON','2016','LABEL2016']].copy() 
NLST_2016 = NLST[['LAT','LON','2016','LABEL2016']].copy() 
NDVI_2016_Urban = NDVI_2016[(NDVI_2016['LABEL2016'] == 4)]
RD_2016_Urban = RD_2016[(RD_2016['LABEL2016'] == 4)]
NTL_2016_Urban = NTL_2016[(NTL_2016['LABEL2016'] == 4)]
NLST_2016_Urban = NLST_2016[(NLST_2016['LABEL2016'] == 4)]
NDVI_2016_Water = NDVI_2016[(NDVI_2016['LABEL2016'] == 1)]
RD_2016_Water = RD_2016[(RD_2016['LABEL2016'] == 1)]
NTL_2016_Water = NTL_2016[(NTL_2016['LABEL2016'] == 1)]
NLST_2016_Water = NLST_2016[(NLST_2016['LABEL2016'] == 1)]
NDVI_2016_NotUrban = NDVI_2016[(NDVI_2016['LABEL2016'] != 4)]
RD_2016_NotUrban = RD_2016[(RD_2016['LABEL2016'] != 4)]
NTL_2016_NotUrban = NTL_2016[(NTL_2016['LABEL2016'] != 4)]
NLST_2016_NotUrban = NLST_2016[(NLST_2016['LABEL2016'] != 4)]
NDVI_2016_NotUrbanNotWater = NDVI_2016_NotUrban[(NDVI_2016_NotUrban['LABEL2016'] != 1)]
RD_2016_NotUrbanNotWater = RD_2016_NotUrban[(RD_2016_NotUrban['LABEL2016'] != 1)]
NTL_2016_NotUrbanNotWater = NTL_2016_NotUrban[(NTL_2016_NotUrban['LABEL2016'] != 1)]
NLST_2016_NotUrbanNotWater = NLST_2016_NotUrban[(NLST_2016_NotUrban['LABEL2016'] != 1)]

NDVI_2017 = NDVI[['LAT','LON','2017','LABEL2017']].copy() 
RD_2017 = RD[['LAT','LON','2017','LABEL2017']].copy() 
NTL_2017 = NTL[['LAT','LON','2017','LABEL2017']].copy() 
NLST_2017 = NLST[['LAT','LON','2017','LABEL2017']].copy() 
NDVI_2017_Urban = NDVI_2017[(NDVI_2017['LABEL2017'] == 4)]
RD_2017_Urban = RD_2017[(RD_2017['LABEL2017'] == 4)]
NTL_2017_Urban = NTL_2017[(NTL_2017['LABEL2017'] == 4)]
NLST_2017_Urban = NLST_2017[(NLST_2017['LABEL2017'] == 4)]
NDVI_2017_Water = NDVI_2017[(NDVI_2017['LABEL2017'] == 1)]
RD_2017_Water = RD_2017[(RD_2017['LABEL2017'] == 1)]
NTL_2017_Water = NTL_2017[(NTL_2017['LABEL2017'] == 1)]
NLST_2017_Water = NLST_2017[(NLST_2017['LABEL2017'] == 1)]
NDVI_2017_NotUrban = NDVI_2017[(NDVI_2017['LABEL2017'] != 4)]
RD_2017_NotUrban = RD_2017[(RD_2017['LABEL2017'] != 4)]
NTL_2017_NotUrban = NTL_2017[(NTL_2017['LABEL2017'] != 4)]
NLST_2017_NotUrban = NLST_2017[(NLST_2017['LABEL2017'] != 4)]
NDVI_2017_NotUrbanNotWater = NDVI_2017_NotUrban[(NDVI_2017_NotUrban['LABEL2017'] != 1)]
RD_2017_NotUrbanNotWater = RD_2017_NotUrban[(RD_2017_NotUrban['LABEL2017'] != 1)]
NTL_2017_NotUrbanNotWater = NTL_2017_NotUrban[(NTL_2017_NotUrban['LABEL2017'] != 1)]
NLST_2017_NotUrbanNotWater = NLST_2017_NotUrban[(NLST_2017_NotUrban['LABEL2017'] != 1)]

NDVI_2018 = NDVI[['LAT','LON','2018','LABEL2018']].copy() 
RD_2018 = RD[['LAT','LON','2018','LABEL2018']].copy() 
NTL_2018 = NTL[['LAT','LON','2018','LABEL2018']].copy() 
NLST_2018 = NLST[['LAT','LON','2018','LABEL2018']].copy() 
NDVI_2018_Urban = NDVI_2018[(NDVI_2018['LABEL2018'] == 4)]
RD_2018_Urban = RD_2018[(RD_2018['LABEL2018'] == 4)]
NTL_2018_Urban = NTL_2018[(NTL_2018['LABEL2018'] == 4)]
NLST_2018_Urban = NLST_2018[(NLST_2018['LABEL2018'] == 4)]
NDVI_2018_Water = NDVI_2018[(NDVI_2018['LABEL2018'] == 1)]
RD_2018_Water = RD_2018[(RD_2018['LABEL2018'] == 1)]
NTL_2018_Water = NTL_2018[(NTL_2018['LABEL2018'] == 1)]
NLST_2018_Water = NLST_2018[(NLST_2018['LABEL2018'] == 1)]
NDVI_2018_NotUrban = NDVI_2018[(NDVI_2018['LABEL2018'] != 4)]
RD_2018_NotUrban = RD_2018[(RD_2018['LABEL2018'] != 4)]
NTL_2018_NotUrban = NTL_2018[(NTL_2018['LABEL2018'] != 4)]
NLST_2018_NotUrban = NLST_2018[(NLST_2018['LABEL2018'] != 4)]
NDVI_2018_NotUrbanNotWater = NDVI_2018_NotUrban[(NDVI_2018_NotUrban['LABEL2018'] != 1)]
RD_2018_NotUrbanNotWater = RD_2018_NotUrban[(RD_2018_NotUrban['LABEL2018'] != 1)]
NTL_2018_NotUrbanNotWater = NTL_2018_NotUrban[(NTL_2018_NotUrban['LABEL2018'] != 1)]
NLST_2018_NotUrbanNotWater = NLST_2018_NotUrban[(NLST_2018_NotUrban['LABEL2018'] != 1)]

NDVI_2019 = NDVI[['LAT','LON','2019','LABEL2019']].copy() 
RD_2019 = RD[['LAT','LON','2019','LABEL2019']].copy() 
NTL_2019 = NTL[['LAT','LON','2019','LABEL2019']].copy() 
NLST_2019 = NLST[['LAT','LON','2019','LABEL2019']].copy() 
NDVI_2019_Urban = NDVI_2019[(NDVI_2019['LABEL2019'] == 4)]
RD_2019_Urban = RD_2019[(RD_2019['LABEL2019'] == 4)]
NTL_2019_Urban = NTL_2019[(NTL_2019['LABEL2019'] == 4)]
NLST_2019_Urban = NLST_2019[(NLST_2019['LABEL2019'] == 4)]
NDVI_2019_Water = NDVI_2019[(NDVI_2019['LABEL2019'] == 1)]
RD_2019_Water = RD_2019[(RD_2019['LABEL2019'] == 1)]
NTL_2019_Water = NTL_2019[(NTL_2019['LABEL2019'] == 1)]
NLST_2019_Water = NLST_2019[(NLST_2019['LABEL2019'] == 1)]
NDVI_2019_NotUrban = NDVI_2019[(NDVI_2019['LABEL2019'] != 4)]
RD_2019_NotUrban = RD_2019[(RD_2019['LABEL2019'] != 4)]
NTL_2019_NotUrban = NTL_2019[(NTL_2019['LABEL2019'] != 4)]
NLST_2019_NotUrban = NLST_2019[(NLST_2019['LABEL2019'] != 4)]
NDVI_2019_NotUrbanNotWater = NDVI_2019_NotUrban[(NDVI_2019_NotUrban['LABEL2019'] != 1)]
RD_2019_NotUrbanNotWater = RD_2019_NotUrban[(RD_2019_NotUrban['LABEL2019'] != 1)]
NTL_2019_NotUrbanNotWater = NTL_2019_NotUrban[(NTL_2019_NotUrban['LABEL2019'] != 1)]
NLST_2019_NotUrbanNotWater = NLST_2019_NotUrban[(NLST_2019_NotUrban['LABEL2019'] != 1)]

NDVI_2020 = NDVI[['LAT','LON','2020','LABEL2020']].copy() 
RD_2020 = RD[['LAT','LON','2020','LABEL2020']].copy() 
NTL_2020 = NTL[['LAT','LON','2020','LABEL2020']].copy() 
NLST_2020 = NLST[['LAT','LON','2020','LABEL2020']].copy() 
NDVI_2020_Urban = NDVI_2020[(NDVI_2020['LABEL2020'] == 4)]
RD_2020_Urban = RD_2020[(RD_2020['LABEL2020'] == 4)]
NTL_2020_Urban = NTL_2020[(NTL_2020['LABEL2020'] == 4)]
NLST_2020_Urban = NLST_2020[(NLST_2020['LABEL2020'] == 4)]
NDVI_2020_Water = NDVI_2020[(NDVI_2020['LABEL2020'] == 1)]
RD_2020_Water = RD_2020[(RD_2020['LABEL2020'] == 1)]
NTL_2020_Water = NTL_2020[(NTL_2020['LABEL2020'] == 1)]
NLST_2020_Water = NLST_2020[(NLST_2020['LABEL2020'] == 1)]
NDVI_2020_NotUrban = NDVI_2020[(NDVI_2020['LABEL2020'] != 4)]
RD_2020_NotUrban = RD_2020[(RD_2020['LABEL2020'] != 4)]
NTL_2020_NotUrban = NTL_2020[(NTL_2020['LABEL2020'] != 4)]
NLST_2020_NotUrban = NLST_2020[(NLST_2020['LABEL2020'] != 4)]
NDVI_2020_NotUrbanNotWater = NDVI_2020_NotUrban[(NDVI_2020_NotUrban['LABEL2020'] != 1)]
RD_2020_NotUrbanNotWater = RD_2020_NotUrban[(RD_2020_NotUrban['LABEL2020'] != 1)]
NTL_2020_NotUrbanNotWater = NTL_2020_NotUrban[(NTL_2020_NotUrban['LABEL2020'] != 1)]
NLST_2020_NotUrbanNotWater = NLST_2020_NotUrban[(NLST_2020_NotUrban['LABEL2020'] != 1)]

In [43]:
def percentileValue(data, year, percentile):
    
    year_column = str(year)
    year_data = data[year_column]
    
    
    year_percentile = np.percentile(year_data, percentile)
    
    return year_percentile
percentileValue(NTL_2020_NotUrbanNotWater, 2020, 95)

10.412931999999994

In [5]:
RD2013_Stats = np.array([0,0,0,0,0,0,558.041960,1112.438963,6247.438480])
RD2014_Stats = np.array([0,0,0,0,0,0,978.329410,2563.660348,8735.687500])
RD2015_Stats = np.array([0,0,0,0,0,376.803450,1408.428162,3396.927272,8755.502930])
RD2016_Stats = np.array([0,0,0,0,0,556.585085,2606.148110,5157.787265,10251.362300])
RD2017_Stats = np.array([0,0,0,0,0,599.366730,3140.882935,5780.547115,10251.362300])
RD2018_Stats = np.array([0,0,0,0,0,606.262080,3168.287352,5884.176035,10251.362300])
RD2019_Stats = np.array([0,0,0,0,0,625.921232,3167.780704,5889.033171,10251.362300])
RD2020_Stats = np.array([0,0,0,0,0,655.783690,3229.035400,5946.476814,10391.162110])

NDVI2013_Stats = np.array([0.006310,0.329295,0.368230,0.406575,0.428990,0.451680,0.497855,0.549734,0.676060])
NDVI2014_Stats = np.array([0.01534,0.31944,0.35232,0.39562,0.42403,0.45191,0.50575,0.57044,0.68919])
NDVI2015_Stats = np.array([0.001440,0.313796,0.339270,0.377435,0.404515,0.433180,0.487600,0.553323,0.690870])
NDVI2016_Stats = np.array([0.084000,0.298626,0.326649,0.360650,0.382765,0.406303,0.459164,0.522264,0.68354])
NDVI2017_Stats = np.array([0.016320,0.314050,0.354115,0.394945,0.423150,0.453055,0.505440,0.562915,0.692900])
NDVI2018_Stats = np.array([0.028490,0.290248,0.325766,0.366240,0.390020,0.414450,0.466064,0.539572,0.693560])
NDVI2019_Stats = np.array([0.034230,0.283785,0.322558,0.362320,0.385895,0.410930,0.464591,0.532113,0.701080])
NDVI2020_Stats = np.array([0.124510,0.317275,0.369860,0.416120,0.443110,0.470660,0.526102,0.578686,0.701330])

NTL2013_Stats = np.array([0.052080,0.117029,0.220620,0.474755,0.898780,1.922630,6.962390,15.433894,166.173920])
NTL2014_Stats = np.array([0.121830,0.189728,0.316550,0.615650,1.086410,2.250460,8.168370,18.230936,139.583240])
NTL2015_Stats = np.array([0.120000,0.226231,0.338837,0.617167,1.059640,2.221200,8.209082,18.110648,116.407750])
NTL2016_Stats = np.array([0.065810,0.145933,0.262828,0.538498,0.980750,2.151752,8.469348,19.178180,133.405980])
NTL2017_Stats = np.array([0.266930,0.339335,0.459495,0.765120,1.249190,2.520820,9.396260,20.549545,113.043390])
NTL2018_Stats = np.array([0.288410,0.383675,0.505946,0.819290,1.309870,2.593840,9.207640,19.376706,79.393230])
NTL2019_Stats = np.array([0.264740,0.361753,0.562240,0.945805,1.563625,3.146050,10.597157,21.035070,104.566140])
NTL2020_Stats = np.array([0.324180,0.436717,0.591118,0.958160,1.515770,2.978240,10.412932,20.602327,106.105190])

NLST2013_Stats = np.array([287.285110,288.048590,288.656923,289.372720,289.908410,290.732959,291.660587,292.370998,294.589492])
NLST2014_Stats = np.array([287.704293,288.619973,289.154898,289.895568,290.460287,291.238315,292.223480,292.973070,295.166597])
NLST2015_Stats = np.array([287.714853,288.427444,288.867664,289.549443,290.218842,290.890540,291.775363,292.491522,294.795607])
NLST2016_Stats = np.array([287.127640,287.982581,288.652156,289.380648,290.043414,290.859635,291.870475,292.733057,295.083667])
NLST2017_Stats = np.array([287.135442,288.091897,288.781463,289.498207,290.071140,290.849413,291.895694,292.797523,295.047878])
NLST2018_Stats = np.array([241.472825,289.383476,289.920992,290.616288,291.038900,291.675645,292.504277,293.060004,294.953475])
NLST2019_Stats = np.array([288.702422,289.265219,289.739503,290.377565,290.835513,291.485930,292.262664,292.898314,294.806772])
NLST2020_Stats = np.array([288.064313,288.709629,289.291594,290.011147,290.526907,291.117690,291.991338,293.069216,296.132568])

In [6]:
Rule1_2013_NotUrbanNotWater = RD_2013_NotUrbanNotWater[['LAT','LON']].copy()
Rule1_2013_NotUrbanNotWater['NTL'] = NTL_2013_NotUrbanNotWater['2013'].values
Rule1_2013_NotUrbanNotWater['NLST'] = NLST_2013_NotUrbanNotWater['2013'].values
Rule1_2013_NotUrbanNotWater['NDVI'] = NDVI_2013_NotUrbanNotWater['2013'].values
Rule1_2013_NotUrbanNotWater['LABEL'] = NTL_2013_NotUrbanNotWater['LABEL2013'].values
Rule1_2013_Urban = RD_2013_Urban[['LAT','LON']].copy()
Rule1_2013_Urban['NTL'] = NTL_2013_Urban['2013'].values
Rule1_2013_Urban['NLST'] = NLST_2013_Urban['2013'].values
Rule1_2013_Urban['NDVI'] = NDVI_2013_Urban['2013'].values
Rule1_2013_Urban['LABEL'] = NTL_2013_Urban['LABEL2013'].values
Rule1_2013_Water = RD_2013_Water[['LAT','LON']].copy()
Rule1_2013_Water['NTL'] = NTL_2013_Water['2013'].values
Rule1_2013_Water['NLST'] = NLST_2013_Water['2013'].values
Rule1_2013_Water['NDVI'] = NDVI_2013_Water['2013'].values
Rule1_2013_Water['LABEL'] = NTL_2013_Water['LABEL2013'].values


Rule1_2014_NotUrbanNotWater = RD_2014_NotUrbanNotWater[['LAT','LON']].copy()
Rule1_2014_NotUrbanNotWater['NTL'] = NTL_2014_NotUrbanNotWater['2014'].values
Rule1_2014_NotUrbanNotWater['NLST'] = NLST_2014_NotUrbanNotWater['2014'].values
Rule1_2014_NotUrbanNotWater['NDVI'] = NDVI_2014_NotUrbanNotWater['2014'].values
Rule1_2014_NotUrbanNotWater['LABEL'] = NTL_2014_NotUrbanNotWater['LABEL2014'].values
Rule1_2014_Urban = RD_2014_Urban[['LAT','LON']].copy()
Rule1_2014_Urban['NTL'] = NTL_2014_Urban['2014'].values
Rule1_2014_Urban['NLST'] = NLST_2014_Urban['2014'].values
Rule1_2014_Urban['NDVI'] = NDVI_2014_Urban['2014'].values
Rule1_2014_Urban['LABEL'] = NTL_2014_Urban['LABEL2014'].values
Rule1_2014_Water = RD_2014_Water[['LAT','LON']].copy()
Rule1_2014_Water['NTL'] = NTL_2014_Water['2014'].values
Rule1_2014_Water['NLST'] = NLST_2014_Water['2014'].values
Rule1_2014_Water['NDVI'] = NDVI_2014_Water['2014'].values
Rule1_2014_Water['LABEL'] = NTL_2014_Water['LABEL2014'].values

Rule1_2015_NotUrbanNotWater = RD_2015_NotUrbanNotWater[['LAT','LON']].copy()
Rule1_2015_NotUrbanNotWater['NTL'] = NTL_2015_NotUrbanNotWater['2015'].values
Rule1_2015_NotUrbanNotWater['NLST'] = NLST_2015_NotUrbanNotWater['2015'].values
Rule1_2015_NotUrbanNotWater['NDVI'] = NDVI_2015_NotUrbanNotWater['2015'].values
Rule1_2015_NotUrbanNotWater['LABEL'] = NTL_2015_NotUrbanNotWater['LABEL2015'].values
Rule1_2015_Urban = RD_2015_Urban[['LAT','LON']].copy()
Rule1_2015_Urban['NTL'] = NTL_2015_Urban['2015'].values
Rule1_2015_Urban['NLST'] = NLST_2015_Urban['2015'].values
Rule1_2015_Urban['NDVI'] = NDVI_2015_Urban['2015'].values
Rule1_2015_Urban['LABEL'] = NTL_2015_Urban['LABEL2015'].values
Rule1_2015_Water = RD_2015_Water[['LAT','LON']].copy()
Rule1_2015_Water['NTL'] = NTL_2015_Water['2015'].values
Rule1_2015_Water['NLST'] = NLST_2015_Water['2015'].values
Rule1_2015_Water['NDVI'] = NDVI_2015_Water['2015'].values
Rule1_2015_Water['LABEL'] = NTL_2015_Water['LABEL2015'].values

Rule1_2016_NotUrbanNotWater = RD_2016_NotUrbanNotWater[['LAT','LON']].copy()
Rule1_2016_NotUrbanNotWater['NTL'] = NTL_2016_NotUrbanNotWater['2016'].values
Rule1_2016_NotUrbanNotWater['NLST'] = NLST_2016_NotUrbanNotWater['2016'].values
Rule1_2016_NotUrbanNotWater['NDVI'] = NDVI_2016_NotUrbanNotWater['2016'].values
Rule1_2016_NotUrbanNotWater['LABEL'] = NTL_2016_NotUrbanNotWater['LABEL2016'].values
Rule1_2016_Urban = RD_2016_Urban[['LAT','LON']].copy()
Rule1_2016_Urban['NTL'] = NTL_2016_Urban['2016'].values
Rule1_2016_Urban['NLST'] = NLST_2016_Urban['2016'].values
Rule1_2016_Urban['NDVI'] = NDVI_2016_Urban['2016'].values
Rule1_2016_Urban['LABEL'] = NTL_2016_Urban['LABEL2016'].values
Rule1_2016_Water = RD_2016_Water[['LAT','LON']].copy()
Rule1_2016_Water['NTL'] = NTL_2016_Water['2016'].values
Rule1_2016_Water['NLST'] = NLST_2016_Water['2016'].values
Rule1_2016_Water['NDVI'] = NDVI_2016_Water['2016'].values
Rule1_2016_Water['LABEL'] = NTL_2016_Water['LABEL2016'].values

Rule1_2017_NotUrbanNotWater = RD_2017_NotUrbanNotWater[['LAT','LON']].copy()
Rule1_2017_NotUrbanNotWater['NTL'] = NTL_2017_NotUrbanNotWater['2017'].values
Rule1_2017_NotUrbanNotWater['NLST'] = NLST_2017_NotUrbanNotWater['2017'].values
Rule1_2017_NotUrbanNotWater['NDVI'] = NDVI_2017_NotUrbanNotWater['2017'].values
Rule1_2017_NotUrbanNotWater['LABEL'] = NTL_2017_NotUrbanNotWater['LABEL2017'].values
Rule1_2017_Urban = RD_2017_Urban[['LAT','LON']].copy()
Rule1_2017_Urban['NTL'] = NTL_2017_Urban['2017'].values
Rule1_2017_Urban['NLST'] = NLST_2017_Urban['2017'].values
Rule1_2017_Urban['NDVI'] = NDVI_2017_Urban['2017'].values
Rule1_2017_Urban['LABEL'] = NTL_2017_Urban['LABEL2017'].values
Rule1_2017_Water = RD_2017_Water[['LAT','LON']].copy()
Rule1_2017_Water['NTL'] = NTL_2017_Water['2017'].values
Rule1_2017_Water['NLST'] = NLST_2017_Water['2017'].values
Rule1_2017_Water['NDVI'] = NDVI_2017_Water['2017'].values
Rule1_2017_Water['LABEL'] = NTL_2017_Water['LABEL2017'].values

Rule1_2018_NotUrbanNotWater = RD_2018_NotUrbanNotWater[['LAT','LON']].copy()
Rule1_2018_NotUrbanNotWater['NTL'] = NTL_2018_NotUrbanNotWater['2018'].values
Rule1_2018_NotUrbanNotWater['NLST'] = NLST_2018_NotUrbanNotWater['2018'].values
Rule1_2018_NotUrbanNotWater['NDVI'] = NDVI_2018_NotUrbanNotWater['2018'].values
Rule1_2018_NotUrbanNotWater['LABEL'] = NTL_2018_NotUrbanNotWater['LABEL2018'].values
Rule1_2018_Urban = RD_2018_Urban[['LAT','LON']].copy()
Rule1_2018_Urban['NTL'] = NTL_2018_Urban['2018'].values
Rule1_2018_Urban['NLST'] = NLST_2018_Urban['2018'].values
Rule1_2018_Urban['NDVI'] = NDVI_2018_Urban['2018'].values
Rule1_2018_Urban['LABEL'] = NTL_2018_Urban['LABEL2018'].values
Rule1_2018_Water = RD_2018_Water[['LAT','LON']].copy()
Rule1_2018_Water['NTL'] = NTL_2018_Water['2018'].values
Rule1_2018_Water['NLST'] = NLST_2018_Water['2018'].values
Rule1_2018_Water['NDVI'] = NDVI_2018_Water['2018'].values
Rule1_2018_Water['LABEL'] = NTL_2018_Water['LABEL2018'].values

Rule1_2019_NotUrbanNotWater = RD_2019_NotUrbanNotWater[['LAT','LON']].copy()
Rule1_2019_NotUrbanNotWater['NTL'] = NTL_2019_NotUrbanNotWater['2019'].values
Rule1_2019_NotUrbanNotWater['NLST'] = NLST_2019_NotUrbanNotWater['2019'].values
Rule1_2019_NotUrbanNotWater['NDVI'] = NDVI_2019_NotUrbanNotWater['2019'].values
Rule1_2019_NotUrbanNotWater['LABEL'] = NTL_2019_NotUrbanNotWater['LABEL2019'].values
Rule1_2019_Urban = RD_2019_Urban[['LAT','LON']].copy()
Rule1_2019_Urban['NTL'] = NTL_2019_Urban['2019'].values
Rule1_2019_Urban['NLST'] = NLST_2019_Urban['2019'].values
Rule1_2019_Urban['NDVI'] = NDVI_2019_Urban['2019'].values
Rule1_2019_Urban['LABEL'] = NTL_2019_Urban['LABEL2019'].values
Rule1_2019_Water = RD_2019_Water[['LAT','LON']].copy()
Rule1_2019_Water['NTL'] = NTL_2019_Water['2019'].values
Rule1_2019_Water['NLST'] = NLST_2019_Water['2019'].values
Rule1_2019_Water['NDVI'] = NDVI_2019_Water['2019'].values
Rule1_2019_Water['LABEL'] = NTL_2019_Water['LABEL2019'].values

Rule1_2020_NotUrbanNotWater = RD_2020_NotUrbanNotWater[['LAT','LON']].copy()
Rule1_2020_NotUrbanNotWater['NTL'] = NTL_2020_NotUrbanNotWater['2020'].values
Rule1_2020_NotUrbanNotWater['NLST'] = NLST_2020_NotUrbanNotWater['2020'].values
Rule1_2020_NotUrbanNotWater['NDVI'] = NDVI_2020_NotUrbanNotWater['2020'].values
Rule1_2020_NotUrbanNotWater['LABEL'] = NTL_2020_NotUrbanNotWater['LABEL2020'].values
Rule1_2020_Urban = RD_2020_Urban[['LAT','LON']].copy()
Rule1_2020_Urban['NTL'] = NTL_2020_Urban['2020'].values
Rule1_2020_Urban['NLST'] = NLST_2020_Urban['2020'].values
Rule1_2020_Urban['NDVI'] = NDVI_2020_Urban['2020'].values
Rule1_2020_Urban['LABEL'] = NTL_2020_Urban['LABEL2020'].values
Rule1_2020_Water = RD_2020_Water[['LAT','LON']].copy()
Rule1_2020_Water['NTL'] = NTL_2020_Water['2020'].values
Rule1_2020_Water['NLST'] = NLST_2020_Water['2020'].values
Rule1_2020_Water['NDVI'] = NDVI_2020_Water['2020'].values
Rule1_2020_Water['LABEL'] = NTL_2020_Water['LABEL2020'].values

In [7]:
for i in range(len(Rule1_2013_NotUrbanNotWater)):
    if (Rule1_2013_NotUrbanNotWater['NLST'].iloc[i] >= NLST2013_Stats[6]) & (Rule1_2013_NotUrbanNotWater['NTL'].iloc[i] >= NTL2013_Stats[6]) & (NDVI2013_Stats[3] <= Rule1_2013_NotUrbanNotWater['NDVI'].iloc[i] <= NDVI2013_Stats[6]):
        Rule1_2013_NotUrbanNotWater['LABEL'].iloc[i] = 3
    elif (Rule1_2013_NotUrbanNotWater['NLST'].iloc[i] <= NLST2013_Stats[2]) & (Rule1_2013_NotUrbanNotWater['NTL'].iloc[i] <= NTL2013_Stats[2]) & ((NDVI2013_Stats[3] <= Rule1_2013_NotUrbanNotWater['NDVI'].iloc[i]) |  (Rule1_2013_NotUrbanNotWater['NDVI'].iloc[i] >= NDVI2013_Stats[6])):
        Rule1_2013_NotUrbanNotWater['LABEL'].iloc[i] = 2
for i in range(len(Rule1_2014_NotUrbanNotWater)):
    if (Rule1_2014_NotUrbanNotWater['NLST'].iloc[i] >= NLST2014_Stats[6]) & (Rule1_2014_NotUrbanNotWater['NTL'].iloc[i] >= NTL2014_Stats[6]) & (NDVI2014_Stats[3] <= Rule1_2014_NotUrbanNotWater['NDVI'].iloc[i] <= NDVI2014_Stats[6]):
        Rule1_2014_NotUrbanNotWater['LABEL'].iloc[i] = 3
    elif (Rule1_2014_NotUrbanNotWater['NLST'].iloc[i] <= NLST2014_Stats[2]) & (Rule1_2014_NotUrbanNotWater['NTL'].iloc[i] <= NTL2014_Stats[2]) & ((NDVI2014_Stats[3] <= Rule1_2014_NotUrbanNotWater['NDVI'].iloc[i]) |  (Rule1_2014_NotUrbanNotWater['NDVI'].iloc[i] >= NDVI2014_Stats[6])):
        Rule1_2014_NotUrbanNotWater['LABEL'].iloc[i] = 2
for i in range(len(Rule1_2015_NotUrbanNotWater)):
    if (Rule1_2015_NotUrbanNotWater['NLST'].iloc[i] >= NLST2015_Stats[6]) & (Rule1_2015_NotUrbanNotWater['NTL'].iloc[i] >= NTL2015_Stats[6]) & (NDVI2015_Stats[3] <= Rule1_2015_NotUrbanNotWater['NDVI'].iloc[i] <= NDVI2015_Stats[6]):
        Rule1_2015_NotUrbanNotWater['LABEL'].iloc[i] = 3
    elif (Rule1_2015_NotUrbanNotWater['NLST'].iloc[i] <= NLST2015_Stats[2]) & (Rule1_2015_NotUrbanNotWater['NTL'].iloc[i] <= NTL2015_Stats[2]) & ((NDVI2015_Stats[3] <= Rule1_2015_NotUrbanNotWater['NDVI'].iloc[i]) |  (Rule1_2015_NotUrbanNotWater['NDVI'].iloc[i] >= NDVI2015_Stats[6])):
        Rule1_2015_NotUrbanNotWater['LABEL'].iloc[i] = 2
for i in range(len(Rule1_2016_NotUrbanNotWater)):
    if (Rule1_2016_NotUrbanNotWater['NLST'].iloc[i] >= NLST2016_Stats[6]) & (Rule1_2016_NotUrbanNotWater['NTL'].iloc[i] >= NTL2016_Stats[6]) & (NDVI2016_Stats[3] <= Rule1_2016_NotUrbanNotWater['NDVI'].iloc[i] <= NDVI2016_Stats[6]):
        Rule1_2016_NotUrbanNotWater['LABEL'].iloc[i] = 3
    elif (Rule1_2016_NotUrbanNotWater['NLST'].iloc[i] <= NLST2016_Stats[2]) & (Rule1_2016_NotUrbanNotWater['NTL'].iloc[i] <= NTL2016_Stats[2]) & ((NDVI2016_Stats[3] <= Rule1_2016_NotUrbanNotWater['NDVI'].iloc[i]) |  (Rule1_2016_NotUrbanNotWater['NDVI'].iloc[i] >= NDVI2016_Stats[6])):
        Rule1_2016_NotUrbanNotWater['LABEL'].iloc[i] = 2
for i in range(len(Rule1_2017_NotUrbanNotWater)):
    if (Rule1_2017_NotUrbanNotWater['NLST'].iloc[i] >= NLST2017_Stats[6]) & (Rule1_2017_NotUrbanNotWater['NTL'].iloc[i] >= NTL2017_Stats[6]) & (NDVI2017_Stats[3] <= Rule1_2017_NotUrbanNotWater['NDVI'].iloc[i] <= NDVI2017_Stats[6]):
        Rule1_2017_NotUrbanNotWater['LABEL'].iloc[i] = 3
    elif (Rule1_2017_NotUrbanNotWater['NLST'].iloc[i] <= NLST2017_Stats[2]) & (Rule1_2017_NotUrbanNotWater['NTL'].iloc[i] <= NTL2017_Stats[2]) & ((NDVI2017_Stats[3] <= Rule1_2017_NotUrbanNotWater['NDVI'].iloc[i]) |  (Rule1_2017_NotUrbanNotWater['NDVI'].iloc[i] >= NDVI2017_Stats[6])):
        Rule1_2017_NotUrbanNotWater['LABEL'].iloc[i] = 2
for i in range(len(Rule1_2018_NotUrbanNotWater)):
    if (Rule1_2018_NotUrbanNotWater['NLST'].iloc[i] >= NLST2018_Stats[6]) & (Rule1_2018_NotUrbanNotWater['NTL'].iloc[i] >= NTL2018_Stats[6]) & (NDVI2018_Stats[3] <= Rule1_2018_NotUrbanNotWater['NDVI'].iloc[i] <= NDVI2018_Stats[6]):
        Rule1_2018_NotUrbanNotWater['LABEL'].iloc[i] = 3
    elif (Rule1_2018_NotUrbanNotWater['NLST'].iloc[i] <= NLST2018_Stats[2]) & (Rule1_2018_NotUrbanNotWater['NTL'].iloc[i] <= NTL2018_Stats[2]) & ((NDVI2018_Stats[3] <= Rule1_2018_NotUrbanNotWater['NDVI'].iloc[i]) |  (Rule1_2018_NotUrbanNotWater['NDVI'].iloc[i] >= NDVI2018_Stats[6])):
        Rule1_2018_NotUrbanNotWater['LABEL'].iloc[i] = 2
for i in range(len(Rule1_2019_NotUrbanNotWater)):
    if (Rule1_2019_NotUrbanNotWater['NLST'].iloc[i] >= NLST2019_Stats[6]) & (Rule1_2019_NotUrbanNotWater['NTL'].iloc[i] >= NTL2019_Stats[6]) & (NDVI2019_Stats[3] <= Rule1_2019_NotUrbanNotWater['NDVI'].iloc[i] <= NDVI2019_Stats[6]):
        Rule1_2019_NotUrbanNotWater['LABEL'].iloc[i] = 3
    elif (Rule1_2019_NotUrbanNotWater['NLST'].iloc[i] <= NLST2019_Stats[2]) & (Rule1_2019_NotUrbanNotWater['NTL'].iloc[i] <= NTL2019_Stats[2]) & ((NDVI2019_Stats[3] <= Rule1_2019_NotUrbanNotWater['NDVI'].iloc[i]) |  (Rule1_2019_NotUrbanNotWater['NDVI'].iloc[i] >= NDVI2019_Stats[6])):
        Rule1_2019_NotUrbanNotWater['LABEL'].iloc[i] = 2
for i in range(len(Rule1_2020_NotUrbanNotWater)):
    if (Rule1_2020_NotUrbanNotWater['NLST'].iloc[i] >= NLST2020_Stats[6]) & (Rule1_2020_NotUrbanNotWater['NTL'].iloc[i] >= NTL2020_Stats[6]) & (NDVI2020_Stats[3] <= Rule1_2020_NotUrbanNotWater['NDVI'].iloc[i] <= NDVI2020_Stats[6]):
        Rule1_2020_NotUrbanNotWater['LABEL'].iloc[i] = 3
    elif (Rule1_2020_NotUrbanNotWater['NLST'].iloc[i] <= NLST2020_Stats[2]) & (Rule1_2020_NotUrbanNotWater['NTL'].iloc[i] <= NTL2020_Stats[2]) & ((NDVI2020_Stats[3] <= Rule1_2020_NotUrbanNotWater['NDVI'].iloc[i]) |  (Rule1_2020_NotUrbanNotWater['NDVI'].iloc[i] >= NDVI2020_Stats[6])):
        Rule1_2020_NotUrbanNotWater['LABEL'].iloc[i] = 2

In [8]:
Rule1_2013_WithNone_Pre = Rule1_2013_NotUrbanNotWater.append(Rule1_2013_Urban, ignore_index=True)
Rule1_2013_WithNone = Rule1_2013_WithNone_Pre.append(Rule1_2013_Water, ignore_index=True)
Rule1_2014_WithNone_Pre = Rule1_2014_NotUrbanNotWater.append(Rule1_2014_Urban, ignore_index=True)
Rule1_2014_WithNone = Rule1_2014_WithNone_Pre.append(Rule1_2014_Water, ignore_index=True)
Rule1_2015_WithNone_Pre = Rule1_2015_NotUrbanNotWater.append(Rule1_2015_Urban, ignore_index=True)
Rule1_2015_WithNone = Rule1_2015_WithNone_Pre.append(Rule1_2015_Water, ignore_index=True)
Rule1_2016_WithNone_Pre = Rule1_2016_NotUrbanNotWater.append(Rule1_2016_Urban, ignore_index=True)
Rule1_2016_WithNone = Rule1_2016_WithNone_Pre.append(Rule1_2016_Water, ignore_index=True)
Rule1_2017_WithNone_Pre = Rule1_2017_NotUrbanNotWater.append(Rule1_2017_Urban, ignore_index=True)
Rule1_2017_WithNone = Rule1_2017_WithNone_Pre.append(Rule1_2017_Water, ignore_index=True)
Rule1_2018_WithNone_Pre = Rule1_2018_NotUrbanNotWater.append(Rule1_2018_Urban, ignore_index=True)
Rule1_2018_WithNone = Rule1_2018_WithNone_Pre.append(Rule1_2018_Water, ignore_index=True)
Rule1_2019_WithNone_Pre = Rule1_2019_NotUrbanNotWater.append(Rule1_2019_Urban, ignore_index=True)
Rule1_2019_WithNone = Rule1_2019_WithNone_Pre.append(Rule1_2019_Water, ignore_index=True)
Rule1_2020_WithNone_Pre = Rule1_2020_NotUrbanNotWater.append(Rule1_2020_Urban, ignore_index=True)
Rule1_2020_WithNone = Rule1_2020_WithNone_Pre.append(Rule1_2020_Water, ignore_index=True)

In [9]:
Rule1_2013_WithNone_Model_Plot = Rule1_2013_WithNone[Rule1_2013_WithNone['LABEL'] != 0]
Rule1_2013_WithNone_Model = Rule1_2013_WithNone[(Rule1_2013_WithNone['LABEL'] != 0) & (Rule1_2013_WithNone['LABEL'] != 4) & (Rule1_2013_WithNone['LABEL'] != 1)]
Rule1_2013_WithNone_Predict = Rule1_2013_WithNone[Rule1_2013_WithNone['LABEL'] == 0]
Rule1_2014_WithNone_Model_Plot = Rule1_2014_WithNone[Rule1_2014_WithNone['LABEL'] != 0]
Rule1_2014_WithNone_Model = Rule1_2014_WithNone[(Rule1_2014_WithNone['LABEL'] != 0) & (Rule1_2014_WithNone['LABEL'] != 4) & (Rule1_2014_WithNone['LABEL'] != 1)]
Rule1_2014_WithNone_Predict = Rule1_2014_WithNone[Rule1_2014_WithNone['LABEL'] == 0]
Rule1_2015_WithNone_Model_Plot = Rule1_2015_WithNone[Rule1_2015_WithNone['LABEL'] != 0]
Rule1_2015_WithNone_Model = Rule1_2015_WithNone[(Rule1_2015_WithNone['LABEL'] != 0) & (Rule1_2015_WithNone['LABEL'] != 4) & (Rule1_2015_WithNone['LABEL'] != 1)]
Rule1_2015_WithNone_Predict = Rule1_2015_WithNone[Rule1_2015_WithNone['LABEL'] == 0]
Rule1_2016_WithNone_Model_Plot = Rule1_2016_WithNone[Rule1_2016_WithNone['LABEL'] != 0]
Rule1_2016_WithNone_Model = Rule1_2016_WithNone[(Rule1_2016_WithNone['LABEL'] != 0) & (Rule1_2016_WithNone['LABEL'] != 4) & (Rule1_2016_WithNone['LABEL'] != 1)]
Rule1_2016_WithNone_Predict = Rule1_2016_WithNone[Rule1_2016_WithNone['LABEL'] == 0]
Rule1_2017_WithNone_Model_Plot = Rule1_2017_WithNone[Rule1_2017_WithNone['LABEL'] != 0]
Rule1_2017_WithNone_Model = Rule1_2017_WithNone[(Rule1_2017_WithNone['LABEL'] != 0) & (Rule1_2017_WithNone['LABEL'] != 4) & (Rule1_2017_WithNone['LABEL'] != 1)]
Rule1_2017_WithNone_Predict = Rule1_2017_WithNone[Rule1_2017_WithNone['LABEL'] == 0]
Rule1_2018_WithNone_Model_Plot = Rule1_2018_WithNone[Rule1_2018_WithNone['LABEL'] != 0]
Rule1_2018_WithNone_Model = Rule1_2018_WithNone[(Rule1_2018_WithNone['LABEL'] != 0) & (Rule1_2018_WithNone['LABEL'] != 4) & (Rule1_2018_WithNone['LABEL'] != 1)]
Rule1_2018_WithNone_Predict = Rule1_2018_WithNone[Rule1_2018_WithNone['LABEL'] == 0]
Rule1_2019_WithNone_Model_Plot = Rule1_2019_WithNone[Rule1_2019_WithNone['LABEL'] != 0]
Rule1_2019_WithNone_Model = Rule1_2019_WithNone[(Rule1_2019_WithNone['LABEL'] != 0) & (Rule1_2019_WithNone['LABEL'] != 4) & (Rule1_2019_WithNone['LABEL'] != 1)]
Rule1_2019_WithNone_Predict = Rule1_2019_WithNone[Rule1_2019_WithNone['LABEL'] == 0]
Rule1_2020_WithNone_Model_Plot = Rule1_2020_WithNone[Rule1_2020_WithNone['LABEL'] != 0]
Rule1_2020_WithNone_Model = Rule1_2020_WithNone[(Rule1_2020_WithNone['LABEL'] != 0) & (Rule1_2020_WithNone['LABEL'] != 4) & (Rule1_2020_WithNone['LABEL'] != 1)]
Rule1_2020_WithNone_Predict = Rule1_2020_WithNone[Rule1_2020_WithNone['LABEL'] == 0]

In [13]:
print(Rule1_2013_WithNone.groupby('LABEL').size())
print(Rule1_2014_WithNone.groupby('LABEL').size())
print(Rule1_2015_WithNone.groupby('LABEL').size())
print(Rule1_2016_WithNone.groupby('LABEL').size())
print(Rule1_2017_WithNone.groupby('LABEL').size())
print(Rule1_2018_WithNone.groupby('LABEL').size())
print(Rule1_2019_WithNone.groupby('LABEL').size())
print(Rule1_2020_WithNone.groupby('LABEL').size())

LABEL
0    24031
1       27
2       15
3       65
4     2320
dtype: int64
LABEL
0    23989
1       27
2       16
3       76
4     2350
dtype: int64
LABEL
0    23908
1       22
2       80
3       92
4     2356
dtype: int64
LABEL
0    23921
1       18
2       57
3      102
4     2360
dtype: int64
LABEL
0    23934
1       36
2       38
3       79
4     2371
dtype: int64
LABEL
0    23766
1       49
2      101
3      110
4     2432
dtype: int64
LABEL
0    23775
1       35
2       65
3       98
4     2485
dtype: int64
LABEL
0    23692
1       15
2      120
3       81
4     2550
dtype: int64


In [12]:
print(Rule1_2013_WithNone_Model.groupby('LABEL').size())
print(Rule1_2014_WithNone_Model.groupby('LABEL').size())
print(Rule1_2015_WithNone_Model.groupby('LABEL').size())
print(Rule1_2016_WithNone_Model.groupby('LABEL').size())
print(Rule1_2017_WithNone_Model.groupby('LABEL').size())
print(Rule1_2018_WithNone_Model.groupby('LABEL').size())
print(Rule1_2019_WithNone_Model.groupby('LABEL').size())
print(Rule1_2020_WithNone_Model.groupby('LABEL').size())

LABEL
2    15
3    65
dtype: int64
LABEL
2    16
3    76
dtype: int64
LABEL
2    80
3    92
dtype: int64
LABEL
2     57
3    102
dtype: int64
LABEL
2    38
3    79
dtype: int64
LABEL
2    101
3    110
dtype: int64
LABEL
2    65
3    98
dtype: int64
LABEL
2    120
3     81
dtype: int64


In [14]:
Rule1_Combined_PandR = combined_df = pd.concat([Rule1_2013_WithNone_Model, Rule1_2014_WithNone_Model, Rule1_2015_WithNone_Model, Rule1_2016_WithNone_Model, Rule1_2017_WithNone_Model, Rule1_2018_WithNone_Model, Rule1_2019_WithNone_Model, Rule1_2020_WithNone_Model], ignore_index=True)

In [15]:
print(Rule1_Combined_PandR.groupby('LABEL').size())

LABEL
2    492
3    703
dtype: int64


In [34]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, accuracy_score, recall_score

Rule1_Combined_PandR_np = np.array(Rule1_Combined_PandR)

features = Rule1_Combined_PandR_np[:, :-1]
labels = Rule1_Combined_PandR_np[:, -1]

clf = SVC(kernel="rbf")
clf.fit(features, labels)

scores = cross_val_score(clf, features, labels, cv=5, scoring="f1_macro")

print("Average F2 score:", np.mean(scores))
print("Average recall:", np.mean(recall_score(labels, clf.predict(features), pos_label=2)))
print("Average accuracy:", np.mean(accuracy_score(labels, clf.predict(features))))

conf_matrix = confusion_matrix(labels, clf.predict(features))
print("Confusion Matrix:")
print(conf_matrix)


Average F2 score: 0.9940212632895559
Average recall: 1.0
Average accuracy: 0.9974895397489539
Confusion Matrix:
[[492   0]
 [  3 700]]


In [31]:
features = Rule1_2013_WithNone_Predict[['LAT', 'LON', 'NTL', 'NLST', 'NDVI']]
predictions_2013 = clf.predict(features)
predictions_2013_df = pd.DataFrame(predictions_2013, columns=['LABEL'])
print(predictions_2013_df.groupby('LABEL').size())

/Applications/anaconda3/envs/ML_env/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but SVC was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


LABEL
2.0    23080
3.0      951
dtype: int64


In [32]:
features = Rule1_2020_WithNone_Predict[['LAT', 'LON', 'NTL', 'NLST', 'NDVI']]
predictions_2020 = clf.predict(features)
predictions_2020_df = pd.DataFrame(predictions_2020, columns=['LABEL'])
print(predictions_2020_df.groupby('LABEL').size())

/Applications/anaconda3/envs/ML_env/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but SVC was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


LABEL
2.0    21793
3.0     1899
dtype: int64


In [39]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score, confusion_matrix

Rule1_Combined_PandR_np = np.array(Rule1_Combined_PandR)

features = Rule1_Combined_PandR_np[:, :-1]
labels = Rule1_Combined_PandR_np[:, -1]

# Split the data into training, validation, and testing sets (60% training, 20% validation, 20% testing)
features_train, features_temp, labels_train, labels_temp = train_test_split(features, labels, test_size=0.4, random_state=42)
features_val, features_test, labels_val, labels_test = train_test_split(features_temp, labels_temp, test_size=0.5, random_state=42)

clf = SVC(kernel="rbf")


clf.fit(features_train, labels_train)
val_predictions = clf.predict(features_val)

f2_score_val = f1_score(labels_val, val_predictions, average='macro')
accuracy_val = accuracy_score(labels_val, val_predictions)
recall_val = recall_score(labels_val, val_predictions, pos_label=2)

print("F2 score on validation set:", f2_score_val)
print("Accuracy on validation set:", accuracy_val)
print("Recall on validation set:", recall_val)


combined_features_train = np.concatenate((features_train, features_val))
combined_labels_train = np.concatenate((labels_train, labels_val))
clf.fit(combined_features_train, combined_labels_train)


test_predictions = clf.predict(features_test)

f2_score_test = f1_score(labels_test, test_predictions, average='macro')
accuracy_test = accuracy_score(labels_test, test_predictions)
recall_test = recall_score(labels_test, test_predictions, pos_label=2)

print("F2 score on test set:", f2_score_test)
print("Accuracy on test set:", accuracy_test)
print("Recall on test set:", recall_test)


conf_matrix = confusion_matrix(labels_test, test_predictions)
print("Confusion Matrix:")
print(conf_matrix)

features = Rule1_2013_WithNone_Predict[['LAT', 'LON', 'NTL', 'NLST', 'NDVI']]
predictions_2013 = clf.predict(features)
predictions_2013_df = Rule1_2013_WithNone_Predict[['LAT', 'LON', 'NTL', 'NLST', 'NDVI']].copy()
predictions_2013_df['LABEL'] = predictions_2013
label_frequency = predictions_2013_df.groupby('LABEL').size()
print(label_frequency)

features = Rule1_2020_WithNone_Predict[['LAT', 'LON', 'NTL', 'NLST', 'NDVI']]
predictions_2020 = clf.predict(features)
predictions_2020_df = Rule1_2020_WithNone_Predict[['LAT', 'LON', 'NTL', 'NLST', 'NDVI']].copy()
predictions_2020_df['LABEL'] = predictions_2020
label_frequency = predictions_2020_df.groupby('LABEL').size()
print(label_frequency)


F2 score on validation set: 0.9782747022997909
Accuracy on validation set: 0.9790794979079498
Recall on validation set: 1.0
F2 score on test set: 0.9866033893238167
Accuracy on test set: 0.9874476987447699
Recall on test set: 1.0
Confusion Matrix:
[[ 88   0]
 [  3 148]]


/Applications/anaconda3/envs/ML_env/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but SVC was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


LABEL
2.0    23197
3.0      834
dtype: int64


/Applications/anaconda3/envs/ML_env/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but SVC was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


LABEL
2.0    21980
3.0     1712
dtype: int64
